In [ ]:
from sandag_rsm.data_load.zones import load_mgra_data
from sandag_rsm.data_load.triplist import load_trip_list, trip_mode_shares_by_mgra, trip_mode_shares_by_taz
from sandag_rsm.poi import poi_taz_mgra, attach_poi_taz_skims
from sandag_rsm.zone_agg import aggregate_zones, viewer, viewer2, aggregate_zones_within_districts, merge_zone_data

In [ ]:
data_dir = "../test/data"

In [ ]:
mgra = load_mgra_data(data_dir=data_dir, simplify_tolerance=10, topo=True)

In [ ]:
mgra['taz20'] = mgra.taz % 20

In [ ]:
trips = load_trip_list("trips_sample.pq", data_dir=data_dir)

In [ ]:
tazs = merge_zone_data(mgra, cluster_id="taz")

In [ ]:
trip_mode_shares = trip_mode_shares_by_taz(trips, tazs=tazs.index, mgra_gdf=mgra)

In [ ]:
tazs = tazs.join(trip_mode_shares.add_prefix("modeshare_"), on='taz')

In [ ]:
poi = poi_taz_mgra(mgra)

In [ ]:
poi

In [ ]:
cluster_factors={'popden':1, 'empden':1, 'modeshare_NM':100, 'modeshare_WT':100}

In [ ]:
tazs, cluster_factors = attach_poi_taz_skims(
    tazs,
    "traffic_skims_AM_mini.omx",
    names='AM_SOV_TR_M_TIME',
    poi=poi,
    data_dir=data_dir,
    cluster_factors=cluster_factors,
)

In [ ]:
explicit_agg=[
    571, 588, 606, 
    [143, 270, 15],
]


In [ ]:
d1 = tazs.query("district27 == 1")

In [ ]:
viewer(d1, color='popden', marker_line_width=0)

In [ ]:
viewer(d1, color='outside_pendleton_gate_AM_SOV_TR_M_TIME', marker_line_width=0)

In [ ]:
viewer(d1, color='modeshare_WT', marker_line_width=0)

In [ ]:
# import itertools
# cluster_factors={'popden':1, 'empden':1, 'modeshare_NM':100, 'modeshare_WT':100}
# cluster_factors |= {f"{i}_{j}":1 for i,j in itertools.product(poi.keys(), ['AM_SOV_TR_M_TIME'])}
cluster_factors

In [ ]:
kmeans1 = aggregate_zones(
    d1, 
    cluster_factors=cluster_factors, 
    n_zones=100,
    explicit_agg=explicit_agg,
    explicit_col='taz',
    use_xy=1e-6,
)

In [ ]:
viewer2(edges=kmeans1, colors=d1, color_col='empden')

In [ ]:
from sandag_rsm.zone_agg import aggregate_zones_within_districts

kmeans = aggregate_zones_within_districts(
    tazs, 
    cluster_factors=cluster_factors, 
    n_zones=1000,
    use_xy=1e-6,
    explicit_agg=explicit_agg,
    explicit_col='taz',
)

In [ ]:
kmeans = kmeans.reset_index(drop=True)

In [ ]:
viewer2(edges=kmeans, colors=kmeans, color_col='empden')

In [ ]:
agglom3full = aggregate_zones_within_districts(
    tazs, 
    cluster_factors=cluster_factors, 
    n_zones=2000,
    method='agglom_adj', 
    use_xy=1e-4,
    explicit_agg=explicit_agg,
    explicit_col='taz',
)

In [ ]:
viewer2(edges=agglom3full, colors=agglom3full, color_col='empden')

In [ ]:
viewer2(edges=agglom3full, colors=agglom3full, color_col='popden')